In [1]:
import pandas as pd
import numpy as np

set2 = pd.read_csv('lv2-2412.csv')
set2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 9 columns):
PRD_ID         205 non-null int64
WEIGHT         205 non-null float64
ENERGY_CONS    205 non-null float64
WATER_CONS     205 non-null float64
SPIN_SPEED     205 non-null float64
NOISE          205 non-null float64
MOTOR_TYPE     205 non-null object
MATERIAL       205 non-null object
SCORE          205 non-null int64
dtypes: float64(5), int64(2), object(2)
memory usage: 14.5+ KB


### Question 1

In [2]:
df240 = set2.copy()

1-1

In [3]:
DF40_A = df240[df240['MOTOR_TYPE']=='A'].copy()
DF40_B = df240[df240['MOTOR_TYPE']=='B'].copy()

1-2

In [4]:
list_diff = []
for col in ['ENERGY_CONS','WATER_CONS','NOISE']:
    COR_A = np.abs(DF40_A.loc[:, [col, 'SCORE']].corr().iloc[0, 1])
    COR_B = np.abs(DF40_B.loc[:, [col, 'SCORE']].corr().iloc[0, 1])
    diff00 = np.abs(COR_A - COR_B)
    list_diff.append(diff00)

In [5]:
list_diff

[0.0062077295438360824, 0.33799334516471036, 0.16717218649035495]

1-3

In [6]:
np.sum(list_diff)

0.5113732611989014

Answer 1

In [7]:
res24 = np.sum(list_diff)
ans24 = round(res24, 3)
print(res24)
print('ans24 :', ans24)

0.5113732611989014
ans24 : 0.511


### Question 2

In [8]:
df250 = set2.copy()

2-1

In [9]:
from sklearn.preprocessing import StandardScaler
scl00 = StandardScaler()
list_fts = ['WEIGHT','ENERGY_CONS','WATER_CONS','SPIN_SPEED','NOISE']
list_fts_s = []
for col in list_fts:
    list_fts_s.append(col + '_S')
scl00.fit(df250.loc[:, list_fts])
df_tmp = pd.DataFrame(scl00.transform(df250.loc[:, list_fts]), \
            columns = list_fts_s)

In [10]:
df_tmp.index = df250.index
df251 = pd.concat([df250, df_tmp], axis=1)
df251

,PRD_ID,WEIGHT,ENERGY_CONS,WATER_CONS,SPIN_SPEED,NOISE,MOTOR_TYPE,MATERIAL,SCORE,WEIGHT_S,ENERGY_CONS_S,WATER_CONS_S,SPIN_SPEED_S,NOISE_S
0,1,251.35,209.10,26.55,1238.97,79.68,A,A,68,-0.306208,0.262800,-0.914288,-1.852161,0.543525
1,2,250.84,208.93,29.32,1240.25,81.78,A,A,75,-0.355352,0.234060,0.626713,-1.823603,1.193455
2,3,254.01,215.46,29.28,1351.34,68.60,A,A,72,-0.049891,1.338015,0.604460,0.654864,-2.885626
3,4,258.55,206.91,25.93,1344.86,77.76,B,B,71,0.387583,-0.107439,-1.259206,0.510292,-0.050696
4,5,258.56,207.93,26.26,1343.22,76.79,C,B,75,0.388546,0.065001,-1.075621,0.473703,-0.350901
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,267.11,207.46,28.50,1306.58,84.67,A,B,71,1.212423,-0.014457,0.170532,-0.343752,2.087881
201,202,266.61,217.00,30.17,1307.46,80.70,A,B,78,1.164243,1.598366,1.099583,-0.324118,0.859205
202,203,264.43,211.90,30.08,1268.00,82.26,A,B,77,0.954179,0.736165,1.049514,-1.204488,1.342010
203,204,264.85,206.63,30.88,1344.79,72.09,A,B,79,0.994650,-0.154776,1.494569,0.508730,-1.805505


2-2, 2-3

In [11]:
from sklearn.cluster import KMeans
n_c = 3
km00 = KMeans(random_state=1234, n_clusters=n_c)
km00.fit(df251.loc[:, list_fts_s])
srs00 = pd.Series(km00.labels_)
srs00.index = df251.index
list_mean = []
for c in range(n_c):
    list_mean.append(df251.loc[srs00==c, 'SCORE'].mean())
    
np.max(list_mean)

76.91025641025641

Answer 2

In [12]:
res25 = np.max(list_mean)
ans25 = round(res25, 2)
print(res25)
print('ans25 : ', ans25)

76.91025641025641
ans25 :  76.91


### Question 3

In [13]:
df260 = set2.copy()

3-1

In [14]:
df_dum = pd.get_dummies(df260.loc[:, ['MOTOR_TYPE','MATERIAL']],drop_first=True)
list_dum = list(df_dum.columns)
df261 = pd.concat([df260, df_dum], axis=1)

3-2

In [15]:
train26 = df261[df261['PRD_ID']%5!=0].copy()
test26 = df261[df261['PRD_ID']%5==0].copy()
print(train26.shape, test26.shape)

(164, 12) (41, 12)


3-3

In [16]:
from sklearn.linear_model import LinearRegression
LR00 = LinearRegression()
list_fts = ['WEIGHT','ENERGY_CONS','WATER_CONS','SPIN_SPEED','NOISE'] + list_dum
LR00.fit(train26.loc[:, list_fts], train26['SCORE'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

3-4

In [17]:
arr_pred = LR00.predict(test26.loc[:, list_fts])
arr_act = np.array(test26['SCORE'])
mse = np.mean((arr_pred-arr_act)**2)
mse

21.632757503003123

Answer 3

In [18]:
res26 = mse
ans26 = round(res26, 3)
print(res26)
print('ans26 : ', ans26)

21.632757503003123
ans26 :  21.633


# SUMMARY

In [19]:
print('ans24 : ', ans24)
print('ans25 : ', ans25)
print('ans26 : ', ans26)

ans24 :  0.511
ans25 :  76.91
ans26 :  21.633
